## Using Python in Julia
#### 在Julia裡面使用Python需要透過PyCall

In [ ]:
# using Pkg
# Pkg.add("PyCall")

In [1]:
using PyCall

#### 呼叫python需在程式碼加前綴的macro "py"

In [2]:
py"1+2"

3

In [3]:
py"type(1+2)"

PyObject <class 'int'>

#### 使用三引號可將整段程式碼包裹在字串中，執行完 不會 回傳執行結果

In [4]:
py"""
import numpy as np
np.sin(np.pi)
"""

#### 使用python套件
1. 可以將 julia 物件送到 python 計算

`py"np.sin"` 對應 python 的函式或物件，可以試著將 julia 物件作為參數傳遞給它進行運算。

In [5]:
py"np.sin"(1.0)

0.8414709848078965

In [6]:
py"np.sin"(1//2)

0.479425538604203

In [7]:
py"np.sin"([0.5π, π, 2π])

3-element Array{Float64,1}:
  1.0
  1.2246467991473532e-16
 -2.4492935982947064e-16

2. `pyimport` 可以將 python 套件載入，並對應到一個 julia 的物件。

使用 `pyimport` 可以更方便在 julia 使用 python 的套件。

`np` 會直接對應 python 的套件，並直接存取底下的功能。

In [8]:
np = pyimport("numpy")
np.sin(1)

0.8414709848078965

In [9]:
math = pyimport("math")
math.sin(math.pi / 4)

0.7071067811865476

#### 使用 Python 的 `with`
資源管理理器 `with` 在 python 中使⽤用非常頻繁，我們一樣可以在 julia 中使⽤用，`@pywith` 是其對應的語法。

`pybuiltin` 則是呼叫 python 的內建語法，這邊我們⽤用到 python 的 `open`。

資源的管理在程式設計上是一個很常見的問題，例如管理開啟的檔案等。

最主要的問題點就在於我們必須確保這些開啟的資源在使用完之後，有確實被關閉（或釋放），如果忘記關閉這些資源，就會造成程式執行上的效能問題，甚至出現錯誤。

Python 語言提供了 `with` 這個獨特的語法，可讓程式設計者更容易管理這些開啟的資源，在這樣的語法架構之下，Python 程式會自動進行資源的建立、清理與回收動作，讓程式設計者在使用各種資源時更為方便。

```Python
    with open(filename) as f:
    #...
```

如上方例子，在python中使用 `with` 開啟檔案時，會將開啟的檔案一樣放在 `f` 變數中，但是這個 `f` 只有在這個 `with` 的範圍內可以使用，而離開這個範圍時 `f` 就會自動被關閉，回收相關的資源。

In [10]:
# 開啟一個名為 file.txt的檔案，並寫入 hello
@pywith pybuiltin("open")("file.txt","w") as f begin
    f.write("hello")
end

#### 呼叫Python的物件
如果計算結果儲存在 python 物件中，我們可以直接呼叫 python 物件來取得其中的值。

In [11]:
py"""
import numpy as np
i = np.sin(np.pi)
"""
i = py"i"

1.2246467991473532e-16

In [12]:
i*2

2.4492935982947064e-16

## Using R in Julia
#### 在Julia裡面使用R需要透過RCall

In [ ]:
# Pkg.add("RCall")

In [13]:
using RCall


R version 3.6.2 (2019-12-12) -- "Dark and Stormy Night"
Copyright (C) 2019 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.



#### 呼叫R需在程式碼加前綴的macro "R"

In [14]:
R"1+2"

RObject{RealSxp}
[1] 3


In [15]:
R"class(1+2)"

RObject{StrSxp}
[1] "numeric"


#### 使用三引號可將整段程式碼包裹在字串中，執行完 會 回傳執行結果

In [16]:
R"""
sin(pi)
"""

RObject{RealSxp}
[1] 1.224606e-16


#### 使用R套件
1. 可以將 julia 物件送到 R 計算

`R"sin"` 對應 R 的函式或物件，可以試著將 julia 物件作為參參數傳遞給它進行運算。

In [17]:
R"sin"(1.0)

RObject{RealSxp}
[1] 0.841471


In [18]:
R"sin"(1/2)

RObject{RealSxp}
[1] 0.4794255


In [19]:
R"sin"([0.5π, π, 2π])

RObject{RealSxp}
[1]  1.000000e+00  1.224606e-16 -2.449213e-16


2. `@rimport` 可以將 R 套件載入，並對應到一個 julia 的物件。

使用 `@rimport` 可以更方便在 julia 使用 R 的套件。

`rbase` 會直接對應 R 的套件，並直接存取底下的功能。

In [20]:
@rimport base as rbase

In [21]:
rbase.matrix(1:6, nrow = 2, ncol = 3)

RObject{IntSxp}
     [,1] [,2] [,3]
[1,]    1    3    5
[2,]    2    4    6


#### 呼叫R的物件
在R裡面新增的變數屬於R Object，無法被Julia直接呼叫。

和呼叫python object不同，必須使用`@rget`才能把變數拉進Julia環境

In [22]:
R"tt03 <- data.frame(A = 1:3, B = 4:6, stringsAsFactors = FLASE)"

RObject{VecSxp}
  A B
1 1 4
2 2 5
3 3 6


In [23]:
R"ls()"

RObject{StrSxp}
[1] "tt03"


In [24]:
julia_tt03 = R"tt03"

RObject{VecSxp}
  A B
1 1 4
2 2 5
3 3 6


上面可以看到，即使指定新的變數，他還是R Object，因此直接在julia環境使用會出現錯誤

In [25]:
julia_tt03[1,2]

MethodError: MethodError: no method matching getindex(::Ptr{VecSxp}, ::Int64, ::Int64)
Closest candidates are:
  getindex(::Ptr{S}, ::Integer) where S<:Union{RCall.ExprSxp, VecSxp} at C:\Users\ocean_chou\.julia\packages\RCall\paaBQ\src\methods.jl:178
  getindex(::Ref, !Matched::CartesianIndex{0}) at multidimensional.jl:1804
  getindex(::Ptr{S}, !Matched::AbstractString) where S<:RCall.VectorSxp at C:\Users\ocean_chou\.julia\packages\RCall\paaBQ\src\methods.jl:107
  ...

In [26]:
@rget tt03

,A,B
,Int64,Int64
1,1,4
2,2,5
3,3,6


In [27]:
tt03

,A,B
,Int64,Int64
1,1,4
2,2,5
3,3,6
